In [1]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import pickle
# import import_ipynb
%run model.ipynb import mymodel
%run threshold.ipynb import IGTthreshold

True


In [2]:
# def calculate_t(img, pixel_cnt):
#   boolidx = img<=1
#   imgtmp = img[boolidx]
#   # if imgtmp.shape[0] < pixel_cnt:
#   #   thres = np.sum(imgtmp)/imgtmp.shape[0]
#   # else:
#   #   thres = np.sum(img)/pixel_cnt
#   thres = np.sum(img)/pixel_cnt
#   return thres

# def frequency_grid(img, segment_length):
#   w = img.shape[1]
#   h = img.shape[0]
#   rest_row = h%segment_length
#   rest_col = w%segment_length
#   segment_size = segment_length**2
#   grid_size = (-1*(-w//segment_length), -1*(-h//segment_length))
#   grid = np.zeros((grid_size[1], grid_size[0]))
#   cur_row = 0
#   cur_col = 0
#   for i in range(grid_size[1]):
#     for j in range(grid_size[0]):
#       if i < grid_size[1]-1 and j < grid_size[0]-1:
#         img_seg = img[cur_row:cur_row+segment_length, cur_col:cur_col+segment_length]
#         grid[i, j] = (img_seg==0).sum()/segment_size
#       elif i >= grid_size[1]-1 and j >= grid_size[0]-1:
#         img_seg = img[cur_row:, cur_col:]
#         if rest_row == 0 and rest_col == 0:
#           grid[i, j] = (img_seg==0).sum()/segment_size
#         elif rest_col == 0:
#           grid[i, j] = (img_seg==0).sum()/(segment_length*rest_row)
#         elif rest_row == 0:
#           grid[i, j] = (img_seg==0).sum()/(segment_length*rest_col)
#         else:
#           grid[i, j] = (img_seg==0).sum()/(rest_col*rest_col)
#       elif i >= grid_size[1]-1:
#         img_seg = img[cur_row:, cur_col:segment_length]
#         if rest_row == 0:
#           grid[i, j] = (img_seg==0).sum()/segment_size
#         else:
#           grid[i, j] = (img_seg==0).sum()/(segment_length*rest_row)
#       elif j >= grid_size[0]-1:
#         img_seg = img[cur_row:segment_length, cur_col:]
#         if rest_col == 0:
#           grid[i, j] = (img_seg==0).sum()/segment_size
#         else:
#           grid[i, j] = (img_seg==0).sum()/(segment_length*rest_col)
      
#       cur_col += segment_length
#     cur_row += segment_length
#     cur_col = 0

#   return grid

In [3]:
# def global_threshold(img):
#   pixel_cnt = img.shape[0]*img.shape[1]
#   img_equalized = img.copy()
#   prev_thres = 0
#   iter_cnt = 0
#   while (iter_cnt < 10000):
#     thres = calculate_t(img_equalized, pixel_cnt)
#     # print('thres: ', thres)
#     if prev_thres != 0 and abs(thres - prev_thres) < 0.0001:
#       break
#     img_subtracted = 1 - thres + img_equalized
#     # ret, img_subtracted = cv2.threshold(img_subtracted, 1, 1, cv2.THRESH_TRUNC)
#     e_min = np.amin(img_subtracted)
#     img_equalized = 1 - ((1-img_subtracted) / (1-e_min))
#     prev_thres = thres
#     iter_cnt += 1
#   # print(iter_cnt)
#   # ret, img_equalized = cv2.threshold(img_equalized, 1, 1, cv2.THRESH_TRUNC)
#   return img_equalized

# def local_threshold(img_ori, seg_len, k):
#   img = img_ori.copy()
#   freq_grid = frequency_grid(img, seg_len)
#   m = np.mean(freq_grid)
#   s = np.std(freq_grid)
#   grid_w = freq_grid.shape[1]
#   grid_h = freq_grid.shape[0]
#   cur_row = 0
#   cur_col = 0
#   for i in range(grid_h):
#     for j in range(grid_w):
#       if freq_grid[i, j] > m+k*s:
#         if j < grid_w-1 and i < grid_h-1:
#           img_seg = img[cur_row:cur_row+seg_len, cur_col:cur_col+seg_len]
#           img[cur_row:cur_row+seg_len, cur_col:cur_col+seg_len] = global_threshold(img_seg)
#         elif j >= grid_w-1 and i >= grid_h-1:
#           img_seg = img[cur_row:, cur_col:]
#           img[cur_row:, cur_col:] = global_threshold(img_seg)
#         elif i >= grid_h-1:
#           img_seg = img[cur_row:, cur_col:cur_col+seg_len]
#           img[cur_row:, cur_col:cur_col+seg_len] = global_threshold(img_seg)
#         elif j >= grid_w-1:
#           img_seg = img[cur_row:cur_row+seg_len, cur_col:]
#           img[cur_row:cur_row+seg_len:, cur_col:] = global_threshold(img_seg)

#       cur_col += seg_len
#     cur_row += seg_len
#     cur_col = 0
  
#   return img

In [4]:
# def IGTthreshold(img):
#     img_thres = global_threshold(img)
#     seg_len = int(max(img.shape)/5)
#     img_thres = local_threshold(img_thres, 50, 2)

#     pix_cnt = img.shape[0]*img.shape[1]
#     final_thres = calculate_t(img_thres, pix_cnt)

#     return final_thres, img_thres


In [5]:
def process_img(img_ori):
    img = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY) # turn picture into grayscale
    
    img_w, img_h = img_ori.shape[1], img_ori.shape[0]
    
    img_rescale = img.copy()
    img_rescale = img_rescale / 255
    
    thres, img_rescale = IGTthreshold(img_rescale)

    ret, img_thres = cv2.threshold(img_rescale, thres, 255, cv2.THRESH_BINARY)
    # ret, img_thres = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    img = img_thres.astype('uint8')
    # ret, img = cv2.threshold(img, 0.5*np.amax(img),255,cv2.THRESH_BINARY)

    # img = cv2.erode(img, (5, 5))
    # img = cv2.dilate(img, (5, 5))
    return img

In [6]:
# deal with contours
def get_charlist(img):
    img_w, img_h = img.shape[1], img.shape[0] # width, height
    img_copy = img.copy()
    char_list = []
    char_x = []

    cntrs, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # sort contours in descending order according to their areas
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)

    possible_w = 0 # possible width of a character
    possible_h = 0 # possible height of a character
    for cntr in cntrs:
        x, y, w, h = cv2.boundingRect(cntr)
        if w >= 0.25*img_w:
            continue
        else:
            if possible_w == 0 and possible_h == 0:
                possible_w = w
                possible_h = h
            if h >= 0.75*possible_h:
                possible_w = (possible_w+w)/2
                possible_h = (possible_h+h)/2

                char_pic = img_copy[y:y+h, x:x+w]
                char_list.append(char_pic)
                char_x.append(x)
            else:
                continue
                
    char_list = [c for _, c in sorted(zip(char_x, char_list))]
    return char_list

In [ ]:
# load model
modelpath = 'savedModel/mymodel_bs_256_e_200_lr_0.0001_wd_0.0001'
device = get_device()
model = mymodel().to(device)
model.load_state_dict(torch.load(modelpath))

# target index
tfile = open('targets.pkl', 'rb')
t = pickle.load(tfile)
tfile.close()
k_list = list(t.keys())
l_list = list(t.values())

In [7]:
def get_testdl(char_list):
    x_test = []
    trans = transforms.ToTensor()
    for c in char_list:
        c = cv2.resize(c, (64, 64))
        x_test += [trans(c)]

    test_dl = DataLoader(dataset=x_test, batch_size=1)
    return test_dl

In [8]:
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
def predict(char_list):
    result = ''
    test_dl = get_testdl(char_list)
    for x_test in test_dl:
        x_test = x_test.to(device)
        out = model(x_test)
        y_pred = torch.argmax(out, axis=1)
        y_label = k_list[l_list.index(y_pred)]
        result += y_label

    return result

In [10]:
def output(img_ori):
    # imgpath = 'test_sample.jpg'
    # img_ori, img_plate = readimg(imgpath)
    img_plate = process_img(img_ori)
    char_list = get_charlist(img_plate)
    result = predict(char_list)
    
    return result

In [11]:
# if __name__ == '__main__':
#     img_ori = cv2.imread('car_demo/plate1002.png')
#     result = output(img_ori)
#     print(result)

JVH466
